# Comprehensive Analysis Data 生成ノートブック

Trajectoryファイルから応力-ひずみ解析と破断界面解析を行い、
`comprehensive_analysis_results.csv`を生成する。

## 出力されるCSVの列
- `substrate`: 基板材料 (Al, Al2O3, AlF3)
- `material`: カソード材料
- `pressure_GPa`: 圧縮圧力
- `comp_temp_K`: 圧縮温度
- `high_temp_K`: 高温処理温度
- `youngs_modulus_GPa`: ヤング率
- `tensile_strength_GPa`: 引張強度
- `yield_stress_GPa_0.2offset`: 降伏応力
- `fracture_step`: 破断ステップ
- `fracture_z`: 破断Z座標
- `{元素}_upper`, `{元素}_lower`: 各元素の上下層での原子数

## 1. ライブラリ読み込みと設定

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import re
import glob

from ase.io import Trajectory
from ase import units
from sklearn.linear_model import LinearRegression
from scipy import interpolate

In [ ]:
# ============================================================================
# ★★★ 設定項目（実行前に変更してください）★★★
# ============================================================================

# 入力: Trajectoryファイルのディレクトリ
TENSILE_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/MD/tensile"

# 出力: 結果を保存するディレクトリ
OUTPUT_DIR = Path("./output/comprehensive_analysis_output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 応力-ひずみ曲線の画像保存ディレクトリ
OUTPUT_FIGURE_DIR = OUTPUT_DIR / "stress_strain_curves"
OUTPUT_FIGURE_DIR.mkdir(parents=True, exist_ok=True)

# 出力CSVファイル名
OUTPUT_CSV_NAME = "comprehensive_analysis_results.csv"

print(f"入力ディレクトリ: {TENSILE_DIR}")
print(f"出力ディレクトリ: {OUTPUT_DIR}")

## 2. 破断界面解析モジュール

In [ ]:
def find_fracture_step(
    traj,
    void_size=3.0,
    search_region_ratio=2.0/3.0,
    num_bins=50,
    density_window_bins=10
):
    """
    Trajectoryをスキャンし、指定サイズの真空層（void）を検出する。
    
    Parameters:
        traj: ASE Trajectoryオブジェクト
        void_size: 探す真空層の最小サイズ (Å)
        search_region_ratio: セルの下側から探索する領域の割合
    
    Returns:
        (破断atoms, 破断ステップ, 破断Z座標)
    """
    try:
        if not traj:
            return None, -1, -1

        min_density_region_info = {
            "atoms": None,
            "step_index": -1,
            "fracture_z": -1.0,
            "min_window_sum": float('inf')
        }

        for i, atoms in enumerate(traj):
            cell = atoms.get_cell()
            lz = cell[2, 2]
            z_max_search = lz * search_region_ratio

            positions = atoms.get_positions()
            z_coords = positions[:, 2]
            
            in_region_mask = (z_coords >= 0.0) & (z_coords < z_max_search)
            region_z_coords = z_coords[in_region_mask]

            if len(region_z_coords) < 2:
                continue

            # 明確なギャップの検出
            sorted_z = np.sort(region_z_coords)
            gaps = sorted_z[1:] - sorted_z[:-1]
            if np.any(gaps >= void_size):
                max_gap_index = np.argmax(gaps)
                fracture_z = (sorted_z[max_gap_index] + sorted_z[max_gap_index + 1]) / 2.0
                return atoms, i, fracture_z

        return None, -1, -1

    except Exception as e:
        print(f"[エラー] 破断検出中: {e}")
        return None, -1, -1

In [ ]:
def analyze_distribution_at_fracture(atoms, z_boundary):
    """
    破断瞬間の原子スナップショットを基に、境界面上下の原子分布を解析する。
    """
    symbols = atoms.get_chemical_symbols()
    positions = atoms.get_positions()
    
    elements_to_count = ["Al", "O", "Li", "Mn", "Co", "Ni", "F"]
    results = {elem: {'upper': 0, 'lower': 0, 'total': 0} for elem in elements_to_count}

    for i, symbol in enumerate(symbols):
        if symbol in elements_to_count:
            results[symbol]['total'] += 1
            if positions[i, 2] > z_boundary:
                results[symbol]['upper'] += 1
            else:
                results[symbol]['lower'] += 1
                
    return results

## 3. 応力-ひずみ解析モジュール

In [ ]:
def calculate_yield_stress(strain, stress, youngs_modulus_mpa, offset_strain=0.002):
    """0.2%オフセット法による降伏応力の計算"""
    if np.isnan(youngs_modulus_mpa) or len(strain) < 10:
        return np.nan, None, np.nan
    
    try:
        strain_range = np.linspace(offset_strain, strain.max(), 1000)
        offset_line = youngs_modulus_mpa * (strain_range - offset_strain)
        
        unique_indices = np.unique(strain, return_index=True)[1]
        if len(unique_indices) < 2:
            return np.nan, None, np.nan
        
        interp_func = interpolate.interp1d(
            strain[unique_indices], stress[unique_indices],
            kind='linear', bounds_error=False, fill_value='extrapolate'
        )
        actual_stress_at_offset_range = interp_func(strain_range)
        stress_diff = actual_stress_at_offset_range - offset_line
        
        valid_mask = ~np.isnan(stress_diff)
        if np.sum(valid_mask) < 2:
            return np.nan, None, np.nan
            
        strain_range_valid = strain_range[valid_mask]
        stress_diff_valid = stress_diff[valid_mask]
        sign_changes = np.where(np.diff(np.sign(stress_diff_valid)))[0]
        
        if len(sign_changes) > 0:
            intersection_idx = sign_changes[0]
            intersection_strain = strain_range_valid[intersection_idx]
            yield_stress = interp_func(intersection_strain)
        else:
            min_diff_idx = np.argmin(np.abs(stress_diff_valid))
            intersection_strain = strain_range_valid[min_diff_idx]
            yield_stress = interp_func(intersection_strain)
        
        if yield_stress < 0 or intersection_strain < offset_strain:
            return np.nan, None, np.nan
        
        return yield_stress, offset_line, intersection_strain
    except Exception:
        return np.nan, None, np.nan

In [ ]:
def analyze_stress_strain(traj, fixed_atoms_indices=[]):
    """
    トラジェクトリから応力ひずみ曲線を解析し、機械特性を計算する。
    """
    results = {
        'youngs_modulus_GPa': np.nan,
        'tensile_strength_GPa': np.nan,
        'yield_stress_GPa_0.2offset': np.nan,
        'intersection_strain': np.nan
    }
    
    if len(traj) < 2:
        return results, None

    stresses_from_ase = []
    material_z_lengths = []
    
    initial_atoms = traj[0]
    unfixed_atom_indices_initial = [
        i for i in range(len(initial_atoms)) if i not in fixed_atoms_indices
    ]
    
    if not unfixed_atom_indices_initial:
        return results, None
        
    initial_material_z_thickness = (
        np.max(initial_atoms.positions[unfixed_atom_indices_initial, 2]) -
        np.min(initial_atoms.positions[unfixed_atom_indices_initial, 2])
    )
    
    if initial_material_z_thickness < 1e-6:
        return results, None

    for atoms in traj:
        stress_zz_cell_basis = atoms.get_stress(voigt=False)[2, 2]
        unfixed_atom_indices_current = [
            j for j in range(len(atoms)) if j not in fixed_atoms_indices
        ]
        
        if unfixed_atom_indices_current:
            current_material_z_thickness = (
                np.max(atoms.positions[unfixed_atom_indices_current, 2]) -
                np.min(atoms.positions[unfixed_atom_indices_current, 2])
            )
        else:
            current_material_z_thickness = 0.0

        stresses_from_ase.append(stress_zz_cell_basis)
        material_z_lengths.append(current_material_z_thickness)

    strain = (np.array(material_z_lengths) - initial_material_z_thickness) / initial_material_z_thickness
    stress_GPa = np.array(stresses_from_ase) / units.GPa
    stress_MPa = stress_GPa * 1000

    data = pd.DataFrame({'strain': strain, 'stress': stress_MPa})
    
    # ヤング率計算
    linear_range_data = data[(data['strain'] >= 0.0) & (data['strain'] <= 0.05)]
    youngs_modulus_mpa = np.nan

    if len(linear_range_data) > 1 and np.std(linear_range_data['strain'].values) > 1e-9:
        X = linear_range_data[['strain']]
        y = linear_range_data['stress']
        linear_model = LinearRegression(fit_intercept=False).fit(X, y)
        youngs_modulus_mpa = linear_model.coef_[0]
        results['youngs_modulus_GPa'] = youngs_modulus_mpa / 1000

    if not data.empty:
        results['tensile_strength_GPa'] = data['stress'].max() / 1000

    # 降伏応力計算
    yield_stress_mpa, _, intersection_strain = calculate_yield_stress(
        data['strain'].values, data['stress'].values, youngs_modulus_mpa
    )
    if not np.isnan(yield_stress_mpa):
        results['yield_stress_GPa_0.2offset'] = yield_stress_mpa / 1000
        results['intersection_strain'] = intersection_strain
        
    return results, data

In [ ]:
def plot_stress_strain_curve(data, analysis_results, file_identifier, output_dir):
    """応力-ひずみ曲線をプロットして保存する。"""
    if data is None or data.empty:
        return

    fig_path = output_dir / f"{file_identifier}_ss_curve.png"
    
    plt.figure(figsize=(10, 7))
    stress_gpa = data['stress'] / 1000
    
    plt.scatter(data['strain'], stress_gpa, label='Original Data', alpha=0.3, s=15, color='gray')

    # 移動平均
    if len(data) >= 20:
        moving_avg_gpa = data['stress'].rolling(window=20).mean() / 1000
        plt.plot(data['strain'], moving_avg_gpa, label='Moving Average (20pt)', color='blue')

    # ヤング率のフィット
    youngs_modulus_gpa = analysis_results.get('youngs_modulus_GPa', np.nan)
    if not np.isnan(youngs_modulus_gpa):
        linear_range_strain = data[(data['strain'] >= 0.0) & (data['strain'] <= 0.05)]['strain']
        linear_fit_gpa = youngs_modulus_gpa * linear_range_strain
        plt.plot(linear_range_strain, linear_fit_gpa, 'r-', linewidth=2,
                 label=f"Young's Modulus Fit ({youngs_modulus_gpa:.1f} GPa)")

    plt.title(f'Stress-Strain Curve for {file_identifier}', fontsize=14)
    plt.xlabel('Strain', fontsize=12)
    plt.ylabel('Stress (GPa)', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(fig_path, dpi=150)
    plt.close()

## 4. ファイル名パース関数

In [ ]:
def parse_traj_filename(filename):
    """
    Trajectoryファイル名からパラメータを抽出する。
    
    対応するパターン:
        {substrate}_{material}_P{pressure}_T{comp_temp}K_HT{high_temp}K_*.traj
    
    Returns:
        dict or None
    """
    pattern = re.compile(
        r'([^_]+)_(.*)_P(\d+\.\d+)_T(\d+)K_HT(\d+)K_.*\.traj'
    )
    match = pattern.match(filename)
    
    if not match:
        return None
    
    return {
        'substrate': match.group(1),
        'material': match.group(2),
        'pressure_GPa': float(match.group(3)),
        'comp_temp_K': int(match.group(4)),
        'high_temp_K': int(match.group(5)),
    }

## 5. メイン解析実行

In [ ]:
def run_comprehensive_analysis(tensile_dir, output_dir, output_figure_dir, save_plots=True):
    """
    全てのTrajectoryファイルに対して解析を実行する。
    """
    traj_paths = glob.glob(os.path.join(tensile_dir, '*.traj'))
    
    if not traj_paths:
        print(f"エラー: ディレクトリ '{tensile_dir}' 内に .traj ファイルが見つかりません。")
        return None

    print(f"=== 包括的解析を開始します ===")
    print(f"対象ファイル数: {len(traj_paths)}")
    
    all_results_list = []
    
    for i, traj_path in enumerate(traj_paths):
        filename = os.path.basename(traj_path)
        
        # ファイル名からパラメータを抽出
        params = parse_traj_filename(filename)
        if params is None:
            print(f"[{i+1}/{len(traj_paths)}] スキップ (パターン不一致): {filename}")
            continue
        
        file_identifier = f"{params['substrate']}_{params['material']}_P{params['pressure_GPa']:.3f}_HT{params['high_temp_K']}K"
        print(f"\n[{i+1}/{len(traj_paths)}] 処理中: {file_identifier}")
        
        # Trajectoryを読み込み
        try:
            traj = Trajectory(traj_path)
            if len(traj) == 0:
                print(f"  スキップ: トラジェクトリが空です")
                continue
        except Exception as e:
            print(f"  エラー: {e}")
            continue

        # 1. 応力ひずみ解析
        ss_results, ss_data = analyze_stress_strain(traj)
        if ss_data is not None:
            print(f"  応力ひずみ解析完了: E={ss_results.get('youngs_modulus_GPa', np.nan):.2f} GPa, "
                  f"σ_max={ss_results.get('tensile_strength_GPa', np.nan):.2f} GPa")
            if save_plots:
                plot_stress_strain_curve(ss_data, ss_results, file_identifier, output_figure_dir)

        # 2. 破断界面解析
        fractured_atoms, step_index, z_loc = find_fracture_step(traj)
        fracture_results = {}
        
        if fractured_atoms is not None:
            print(f"  破断検出: ステップ {step_index}, Z={z_loc:.2f} Å")
            dist_results = analyze_distribution_at_fracture(fractured_atoms, z_loc)
            
            fracture_results['fracture_step'] = step_index
            fracture_results['fracture_z'] = z_loc
            for elem, counts in dist_results.items():
                fracture_results[f'{elem}_upper'] = counts['upper']
                fracture_results[f'{elem}_lower'] = counts['lower']
                fracture_results[f'{elem}_total'] = counts['total']
        else:
            print(f"  破断未検出")

        # 結果を統合
        row = params.copy()
        row.update(ss_results)
        row.update(fracture_results)
        all_results_list.append(row)

    print("\n\n=== 解析完了 ===")
    
    if not all_results_list:
        print("有効な解析結果がありませんでした。")
        return None

    # CSVに保存
    df = pd.DataFrame(all_results_list)
    output_csv_path = output_dir / OUTPUT_CSV_NAME
    df.to_csv(output_csv_path, index=False)
    print(f"\n結果を保存: {output_csv_path}")

    # サマリー表示
    print("\n--- 解析結果サマリー ---")
    print(f"処理ファイル数: {len(df)}")
    print(f"平均ヤング率: {df['youngs_modulus_GPa'].mean():.2f} GPa")
    print(f"平均引張強度: {df['tensile_strength_GPa'].mean():.2f} GPa")
    if 'fracture_step' in df.columns:
        print(f"破断検出率: {df['fracture_step'].notna().mean() * 100:.1f}%")

    return df

In [ ]:
# ★★★ 解析を実行 ★★★
df_results = run_comprehensive_analysis(
    tensile_dir=TENSILE_DIR,
    output_dir=OUTPUT_DIR,
    output_figure_dir=OUTPUT_FIGURE_DIR,
    save_plots=True  # Falseにすると応力-ひずみ曲線の保存をスキップ
)

## 6. 結果の確認

In [ ]:
if df_results is not None:
    print("=== 解析結果の先頭10行 ===")
    display_cols = ['substrate', 'material', 'pressure_GPa', 'high_temp_K',
                    'tensile_strength_GPa', 'youngs_modulus_GPa']
    display_cols = [c for c in display_cols if c in df_results.columns]
    print(df_results[display_cols].head(10).to_string())

In [ ]:
if df_results is not None:
    print("\n=== 基板別統計 ===")
    print(df_results.groupby('substrate').agg({
        'tensile_strength_GPa': ['mean', 'std', 'count'],
        'youngs_modulus_GPa': ['mean', 'std']
    }).round(3))

In [ ]:
# 生成されたCSVファイルのパスを表示
output_csv_path = OUTPUT_DIR / OUTPUT_CSV_NAME
print(f"\n生成されたCSVファイル: {output_csv_path}")
print(f"\nこのCSVを al_nmc_contamination_analysis.ipynb で使用してください。")